## ⚙️ 1. Setup: Install the ADK Library

First, let's install the specific version of the Google Agent Development Kit (ADK) that this notebook is built with. Pinning the version ensures our code will always work as expected.

In [ ]:
!pip install google-adk==1.8.0 -q

## 🔑 2. Authentication: Configure Your API Key

Next, we need to securely provide our Google API key. This code will create a secure input prompt for you to paste your key. It then sets the key as an environment variable, which is the standard way the ADK authenticates your requests.

In [ ]:
import os
from getpass import getpass

# Prompt the user for their API key securely
api_key = getpass('Enter your Google API Key: ')

# Set the API key as an environment variable for ADK to use
os.environ['GOOGLE_API_KEY'] = api_key

print("✅ API Key configured successfully! Let the fun begin.")

## 🛠️ 3. Creating Custom Tools

The real power of agents comes from giving them custom skills. Here, we define two simple Python functions, add_guest and get_guest_list, to manage an event's guest list. The ADK is smart enough to use these functions as tools, relying on their names and docstrings to understand what they do.

In [ ]:
import json
# Simple dictionary to act as our guest list database.
GUEST_DATABASE = {}

def add_guest(name: str, email: str) -> str:
  """Adds a guest's name and email to the event's guest list."""
  print(f"Tool executed: Adding guest '{name}' with email '{email}'.")
  GUEST_DATABASE[email] = name
  return f"Successfully added {name} to the guest list."

def get_guest_list() -> str:
  """Retrieves the current list of all registered guests for the event."""
  print("Tool executed: Retrieving guest list.")
  if not GUEST_DATABASE:
    return "The guest list is currently empty."
  return json.dumps(GUEST_DATABASE)

## ⚠️ 4. An Important Lesson: Trying to Mix Tool Types

Our first instinct might be to add our new custom tools directly to the event_planner_agent from Part 1. In this cell, we'll define an agent that attempts to mix the built-in Google Search tool with our new custom functions. As we'll see in the next step, this will fail, but it teaches us an important lesson about agent design.

In [ ]:
from google.adk.agents import Agent
from google.adk.tools import google_search

event_planner_agent = Agent(
    name="event_planner_agent",
    model="gemini-2.5-flash",
    instruction="An agent that plans events and manages guests.",
    # Attempting to mix a built-in tool with custom function tools
    tools=[google_search, add_guest, get_guest_list]
)

## 🚀 5. Building the Execution Engine

This is our helper function to run queries. It handles the core ADK logic: initializing the Runner, streaming events with run_async, and displaying the final response. We'll use this function to interact with all the agents we create.

In [ ]:
from IPython.display import display, Markdown

from google.genai.types import Content, Part
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService, Session

async def run_agent_query(agent: Agent, query: str, session: Session, user_id: str):
    """Initializes a runner and executes a query for a given agent and session."""
    print(f"\n🚀 Running query for agent: '{agent.name}' in session: '{session.id}'...")

    runner = Runner(
        agent=agent,
        session_service=session_service,
        app_name=agent.name
    )

    final_response = ""
    try:
        async for event in runner.run_async(
            user_id=user_id,
            session_id=session.id,
            new_message=Content(parts=[Part(text=query)], role="user")
        ):
            if event.is_final_response():
                final_response = event.content.parts[0].text
    except Exception as e:
        final_response = f"An error occurred: {e}"

    print("\n" + "-"*50)
    print("✅ Final Response:")
    display(Markdown(final_response))
    print("-"*50 + "\n")

    return final_response

# --- Initialize our Session Service ---
session_service = InMemorySessionService()
user_id = "adk_event_planner_001"

## 💥 6. Seeing the Error in Action

Now, let's try to run our event_planner_agent that has mixed tools. Executing this cell will produce an error. This is expected and demonstrates the ADK's design principle that encourages creating specialized agents instead of one agent that does everything.

In [ ]:
# This will NOT work
async def run_event_planner():
    # Create a new, single-use session for this query
    event_planning_session = await session_service.create_session(
        app_name=event_planner_agent.name,
        user_id=user_id
    )

    # Note the new budget constraint in the query!
    query = "Plan a small tech meetup for 30 people in New York on AI/ML"
    print(f"🗣️ User Query: '{query}'")

    await run_agent_query(event_planner_agent, query, event_planning_session, user_id)

await run_event_planner()

## ✅ 7. The Solution: The Specialist Agent

The correct pattern is to create a specialist agent that groups related custom tools. Here, we define the guest_management_agent. Its only job is to be an expert at managing the guest list using the add_guest and get_guest_list tools.

In [ ]:
guest_management_agent = Agent(
    name="guest_management_agent",
    model="gemini-2.5-flash",
    description="A specialized agent for managing an event's guest list. It can add guests and retrieve the current list.",
    instruction="""
    You are a guest management assistant. Your only job is to add guests to a list
    or retrieve the full list when asked.
    - Use the `add_guest` tool to add a new person to the list.
    - Use the `get_guest_list` tool to show the current guest list.
    You do not perform any other tasks like planning events or searching for information.
    Stick strictly to managing the guest list with your tools.
    """,
    tools=[add_guest, get_guest_list]
)
print(f"🧞 Specialist Agent '{guest_management_agent.name}' is created and ready!")

## ✨ 8. Testing Our Specialist Agent

Finally, let's run our new guest_management_agent to prove that it works correctly on its own. We'll send it two queries in the same session to test both of its custom tools and see how it uses its memory.

In [ ]:
async def run_guest_manager():
    guest_management_session = await session_service.create_session(
        app_name=guest_management_agent.name,
        user_id=user_id
    )

    # Run two queries in the same session
    query1 = "Add 'Tim Apple' with email 'tim.a@example.com' to the guest list."
    await run_agent_query(guest_management_agent, query1, guest_management_session, user_id)

    query2 = "Now show me the guest list."
    await run_agent_query(guest_management_agent, query2, guest_management_session, user_id)

await run_guest_manager()